<a href="https://colab.research.google.com/github/Pranada6467/PCAP-project/blob/main/doneinc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%writefile image_processing.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define CHECK_CUDA_ERROR(call) do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        fprintf(stderr, "CUDA Error at %s:%d - %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
        exit(EXIT_FAILURE); \
    } \
} while(0)

// CPU Gaussian Blur
void gaussian_blur_cpu(unsigned char *input, unsigned char *output, int width, int height) {
    float kernel[3][3] = {{1/16.0, 2/16.0, 1/16.0}, {2/16.0, 4/16.0, 2/16.0}, {1/16.0, 2/16.0, 1/16.0}};
    for (int y = 1; y < height - 1; y++) {
        for (int x = 1; x < width - 1; x++) {
            float sum = 0.0;
            for (int ky = -1; ky <= 1; ky++) {
                for (int kx = -1; kx <= 1; kx++) {
                    sum += input[(y + ky) * width + (x + kx)] * kernel[ky + 1][kx + 1];
                }
            }
            output[y * width + x] = (unsigned char)sum;
        }
    }
}

// GPU Gaussian Blur Kernel with Shared Memory
__global__ void gaussian_blur_gpu(unsigned char *input, unsigned char *output, int width, int height) {
    __shared__ unsigned char shared_mem[18][18]; // 16x16 block + 1-pixel halo
    int tx = threadIdx.x, ty = threadIdx.y;
    int x = blockIdx.x * blockDim.x + tx;
    int y = blockIdx.y * blockDim.y + ty;

    // Load data into shared memory (including halo)
    if (x < width && y < height) {
        shared_mem[ty + 1][tx + 1] = input[y * width + x];
        if (ty == 0 && y > 0) shared_mem[0][tx + 1] = input[(y - 1) * width + x];
        if (ty == blockDim.y - 1 && y < height - 1) shared_mem[ty + 2][tx + 1] = input[(y + 1) * width + x];
        if (tx == 0 && x > 0) shared_mem[ty + 1][0] = input[y * width + (x - 1)];
        if (tx == blockDim.x - 1 && x < width - 1) shared_mem[ty + 1][tx + 2] = input[y * width + (x + 1)];
    }
    __syncthreads();

    if (x >= 1 && x < width - 1 && y >= 1 && y < height - 1) {
        float kernel[3][3] = {{1/16.0, 2/16.0, 1/16.0}, {2/16.0, 4/16.0, 2/16.0}, {1/16.0, 2/16.0, 1/16.0}};
        float sum = 0.0;
        for (int ky = -1; ky <= 1; ky++) {
            for (int kx = -1; kx <= 1; kx++) {
                sum += shared_mem[ty + 1 + ky][tx + 1 + kx] * kernel[ky + 1][kx + 1];
            }
        }
        output[y * width + x] = (unsigned char)sum;
    }
}

// CPU Sobel
void sobel_cpu(unsigned char *input, unsigned char *output, int width, int height) {
    int Gx[3][3] = {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}};
    int Gy[3][3] = {{-1, -2, -1}, {0, 0, 0}, {1, 2, 1}};
    for (int y = 1; y < height - 1; y++) {
        for (int x = 1; x < width - 1; x++) {
            float gx = 0, gy = 0;
            for (int ky = -1; ky <= 1; ky++) {
                for (int kx = -1; kx <= 1; kx++) {
                    gx += input[(y + ky) * width + (x + kx)] * Gx[ky + 1][kx + 1];
                    gy += input[(y + ky) * width + (x + kx)] * Gy[ky + 1][kx + 1];
                }
            }
            output[y * width + x] = (unsigned char)(sqrt(gx * gx + gy * gy) > 255 ? 255 : sqrt(gx * gx + gy * gy));
        }
    }
}

// GPU Sobel Kernel
__global__ void sobel_gpu(unsigned char *input, unsigned char *output, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int Gx[3][3] = {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}};
    int Gy[3][3] = {{-1, -2, -1}, {0, 0, 0}, {1, 2, 1}};

    if (x >= 1 && x < width - 1 && y >= 1 && y < height - 1) {
        float gx = 0, gy = 0;
        for (int ky = -1; ky <= 1; ky++) {
            for (int kx = -1; kx <= 1; kx++) {
                gx += input[(y + ky) * width + (x + kx)] * Gx[ky + 1][kx + 1];
                gy += input[(y + ky) * width + (x + kx)] * Gy[ky + 1][kx + 1];
            }
        }
        output[y * width + x] = (unsigned char)(sqrt(gx * gx + gy * gy) > 255 ? 255 : sqrt(gx * gx + gy * gy));
    }
}

// Compare CPU and GPU outputs
int compare_outputs(unsigned char *cpu_output, unsigned char *gpu_output, int size) {
    int differences = 0;
    for (int i = 0; i < size; i++) {
        if (abs(cpu_output[i] - gpu_output[i]) > 1) differences++; // Tolerance of 1
    }
    return differences;
}

int main() {
    int sizes[] = {512, 1024}; // Test multiple image sizes
    const char *techniques[] = {"Gaussian Blur", "Sobel"};
    int num_techniques = 2;

    for (int s = 0; s < 2; s++) {
        int width = sizes[s], height = sizes[s];
        int size = width * height * sizeof(unsigned char);

        // Host memory allocation
        unsigned char *h_input = (unsigned char *)malloc(size);
        unsigned char *h_output_cpu = (unsigned char *)malloc(size);
        unsigned char *h_output_gpu = (unsigned char *)malloc(size);

        // Random input
        srand(time(NULL));
        for (int i = 0; i < width * height; i++) h_input[i] = rand() % 256;

        // Device memory allocation
        unsigned char *d_input, *d_output;
        CHECK_CUDA_ERROR(cudaMalloc(&d_input, size));
        CHECK_CUDA_ERROR(cudaMalloc(&d_output, size));
        CHECK_CUDA_ERROR(cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice));

        // Kernel launch configuration
        dim3 block(16, 16);
        dim3 grid((width + block.x - 1) / block.x, (height + block.y - 1) / block.y);

        // Timing variables
        double cpu_times[num_techniques];
        float gpu_times[num_techniques];
        cudaEvent_t start_gpu, stop_gpu;
        CHECK_CUDA_ERROR(cudaEventCreate(&start_gpu));
        CHECK_CUDA_ERROR(cudaEventCreate(&stop_gpu));

        // Gaussian Blur
        clock_t start = clock();
        gaussian_blur_cpu(h_input, h_output_cpu, width, height);
        clock_t end = clock();
        cpu_times[0] = (double)(end - start) / CLOCKS_PER_SEC * 1000;

        CHECK_CUDA_ERROR(cudaEventRecord(start_gpu));
        gaussian_blur_gpu<<<grid, block>>>(d_input, d_output, width, height);
        CHECK_CUDA_ERROR(cudaEventRecord(stop_gpu));
        CHECK_CUDA_ERROR(cudaEventSynchronize(stop_gpu));
        CHECK_CUDA_ERROR(cudaEventElapsedTime(&gpu_times[0], start_gpu, stop_gpu));
        CHECK_CUDA_ERROR(cudaMemcpy(h_output_gpu, d_output, size, cudaMemcpyDeviceToHost));

        int diff_gaussian = compare_outputs(h_output_cpu, h_output_gpu, size);

        // Sobel
        start = clock();
        sobel_cpu(h_input, h_output_cpu, width, height);
        end = clock();
        cpu_times[1] = (double)(end - start) / CLOCKS_PER_SEC * 1000;

        CHECK_CUDA_ERROR(cudaEventRecord(start_gpu));
        sobel_gpu<<<grid, block>>>(d_input, d_output, width, height);
        CHECK_CUDA_ERROR(cudaEventRecord(stop_gpu));
        CHECK_CUDA_ERROR(cudaEventSynchronize(stop_gpu));
        CHECK_CUDA_ERROR(cudaEventElapsedTime(&gpu_times[1], start_gpu, stop_gpu));
        CHECK_CUDA_ERROR(cudaMemcpy(h_output_gpu, d_output, size, cudaMemcpyDeviceToHost));

        int diff_sobel = compare_outputs(h_output_cpu, h_output_gpu, size);

        // Performance Analysis
        printf("\nImage Size: %dx%d\n", width, height);
        for (int i = 0; i < num_techniques; i++) {
            printf("%s - CPU: %.2f ms, GPU: %.2f ms, Speedup: %.2fx, Differences: %d\n",
                   techniques[i], cpu_times[i], gpu_times[i], cpu_times[i] / gpu_times[i], i == 0 ? diff_gaussian : diff_sobel);
        }

        // Cleanup
        free(h_input); free(h_output_cpu); free(h_output_gpu);
        CHECK_CUDA_ERROR(cudaFree(d_input));
        CHECK_CUDA_ERROR(cudaFree(d_output));
        CHECK_CUDA_ERROR(cudaEventDestroy(start_gpu));
        CHECK_CUDA_ERROR(cudaEventDestroy(stop_gpu));
    }
    return 0;
}


Writing image_processing.cu


In [5]:
!nvcc -o image_processing image_processing.cu -lm  # Link math library for sqrt
!./image_processing


Image Size: 512x512
Gaussian Blur - CPU: 8.73 ms, GPU: 7.78 ms, Speedup: 1.12x, Differences: 260100
Sobel - CPU: 18.75 ms, GPU: 0.00 ms, Speedup: 5006.94x, Differences: 260092

Image Size: 1024x1024
Gaussian Blur - CPU: 33.51 ms, GPU: 0.00 ms, Speedup: 9350.73x, Differences: 1046642
Sobel - CPU: 80.02 ms, GPU: 0.00 ms, Speedup: 17861.61x, Differences: 1046621


Optical Flow

In [6]:
%%writefile optical_flow.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <cuda_runtime.h>

#define CHECK_CUDA_ERROR(call) do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        fprintf(stderr, "CUDA Error at %s:%d - %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
        exit(EXIT_FAILURE); \
    } \
} while(0)

// CPU Optical Flow (Simplified Gradient-Based)
void optical_flow_cpu(unsigned char *frame1, unsigned char *frame2, float *flow_x, float *flow_y, int width, int height) {
    for (int y = 1; y < height - 1; y++) {
        for (int x = 1; x < width - 1; x++) {
            // Spatial gradients (Ix, Iy) from frame1
            float Ix = (float)(frame1[y * width + (x + 1)] - frame1[y * width + (x - 1)]) / 2.0;
            float Iy = (float)(frame1[(y + 1) * width + x] - frame1[(y - 1) * width + x]) / 2.0;
            // Temporal gradient (It) between frame1 and frame2
            float It = (float)(frame2[y * width + x] - frame1[y * width + x]);

            // Solve for flow (dx, dy) using simplified Lucas-Kanade: Ix*dx + Iy*dy = -It
            float denom = Ix * Ix + Iy * Iy + 1e-6; // Avoid division by zero
            flow_x[y * width + x] = -It * Ix / denom;
            flow_y[y * width + x] = -It * Iy / denom;
        }
    }
}

// GPU Optical Flow Kernel
__global__ void optical_flow_gpu(unsigned char *frame1, unsigned char *frame2, float *flow_x, float *flow_y, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= 1 && x < width - 1 && y >= 1 && y < height - 1) {
        // Spatial gradients (Ix, Iy) from frame1
        float Ix = (float)(frame1[y * width + (x + 1)] - frame1[y * width + (x - 1)]) / 2.0;
        float Iy = (float)(frame1[(y + 1) * width + x] - frame1[(y - 1) * width + x]) / 2.0;
        // Temporal gradient (It)
        float It = (float)(frame2[y * width + x] - frame1[y * width + x]);

        // Solve for flow
        float denom = Ix * Ix + Iy * Iy + 1e-6;
        flow_x[y * width + x] = -It * Ix / denom;
        flow_y[y * width + x] = -It * Iy / denom;
    }
}

// Compare CPU and GPU flow outputs (count significant differences)
int compare_flow_outputs(float *cpu_x, float *gpu_x, float *cpu_y, float *gpu_y, int size) {
    int differences = 0;
    for (int i = 0; i < size; i++) {
        if (fabs(cpu_x[i] - gpu_x[i]) > 0.1 || fabs(cpu_y[i] - gpu_y[i]) > 0.1) differences++; // Tolerance of 0.1
    }
    return differences;
}

int main() {
    int sizes[] = {512, 1024}; // Test multiple image sizes
    for (int s = 0; s < 2; s++) {
        int width = sizes[s], height = sizes[s];
        int size = width * height;
        int byte_size = size * sizeof(unsigned char);
        int float_size = size * sizeof(float);

        // Host memory allocation
        unsigned char *h_frame1 = (unsigned char *)malloc(byte_size);
        unsigned char *h_frame2 = (unsigned char *)malloc(byte_size);
        float *h_flow_x_cpu = (float *)malloc(float_size);
        float *h_flow_y_cpu = (float *)malloc(float_size);
        float *h_flow_x_gpu = (float *)malloc(float_size);
        float *h_flow_y_gpu = (float *)malloc(float_size);

        // Generate random frames (grayscale)
        srand(time(NULL));
        for (int i = 0; i < size; i++) {
            h_frame1[i] = rand() % 256;
            h_frame2[i] = h_frame1[i] + (rand() % 11 - 5); // Simulate small motion
        }

        // Device memory allocation
        unsigned char *d_frame1, *d_frame2;
        float *d_flow_x, *d_flow_y;
        CHECK_CUDA_ERROR(cudaMalloc(&d_frame1, byte_size));
        CHECK_CUDA_ERROR(cudaMalloc(&d_frame2, byte_size));
        CHECK_CUDA_ERROR(cudaMalloc(&d_flow_x, float_size));
        CHECK_CUDA_ERROR(cudaMalloc(&d_flow_y, float_size));
        CHECK_CUDA_ERROR(cudaMemcpy(d_frame1, h_frame1, byte_size, cudaMemcpyHostToDevice));
        CHECK_CUDA_ERROR(cudaMemcpy(d_frame2, h_frame2, byte_size, cudaMemcpyHostToDevice));

        // Kernel launch configuration
        dim3 block(16, 16);
        dim3 grid((width + block.x - 1) / block.x, (height + block.y - 1) / block.y);

        // CPU Timing
        clock_t start = clock();
        optical_flow_cpu(h_frame1, h_frame2, h_flow_x_cpu, h_flow_y_cpu, width, height);
        clock_t end = clock();
        double cpu_time = (double)(end - start) / CLOCKS_PER_SEC * 1000;

        // GPU Timing
        cudaEvent_t start_gpu, stop_gpu;
        CHECK_CUDA_ERROR(cudaEventCreate(&start_gpu));
        CHECK_CUDA_ERROR(cudaEventCreate(&stop_gpu));
        CHECK_CUDA_ERROR(cudaEventRecord(start_gpu));
        optical_flow_gpu<<<grid, block>>>(d_frame1, d_frame2, d_flow_x, d_flow_y, width, height);
        CHECK_CUDA_ERROR(cudaEventRecord(stop_gpu));
        CHECK_CUDA_ERROR(cudaEventSynchronize(stop_gpu));
        float gpu_time;
        CHECK_CUDA_ERROR(cudaEventElapsedTime(&gpu_time, start_gpu, stop_gpu));

        // Copy results back
        CHECK_CUDA_ERROR(cudaMemcpy(h_flow_x_gpu, d_flow_x, float_size, cudaMemcpyDeviceToHost));
        CHECK_CUDA_ERROR(cudaMemcpy(h_flow_y_gpu, d_flow_y, float_size, cudaMemcpyDeviceToHost));

        // Validate correctness
        int differences = compare_flow_outputs(h_flow_x_cpu, h_flow_x_gpu, h_flow_y_cpu, h_flow_y_gpu, size);

        // Performance Analysis
        printf("Optical Flow (%dx%d) - CPU: %.2f ms, GPU: %.2f ms, Speedup: %.2fx, Differences: %d\n",
               width, height, cpu_time, gpu_time, cpu_time / gpu_time, differences);

        // Cleanup
        free(h_frame1); free(h_frame2);
        free(h_flow_x_cpu); free(h_flow_y_cpu);
        free(h_flow_x_gpu); free(h_flow_y_gpu);
        CHECK_CUDA_ERROR(cudaFree(d_frame1));
        CHECK_CUDA_ERROR(cudaFree(d_frame2));
        CHECK_CUDA_ERROR(cudaFree(d_flow_x));
        CHECK_CUDA_ERROR(cudaFree(d_flow_y));
        CHECK_CUDA_ERROR(cudaEventDestroy(start_gpu));
        CHECK_CUDA_ERROR(cudaEventDestroy(stop_gpu));
    }
    return 0;
}

Writing optical_flow.cu


In [7]:
!nvcc -o optical_flow optical_flow.cu -lm  # Link math library for fabs, sqrt
!./optical_flow

Optical Flow (512x512) - CPU: 5.48 ms, GPU: 7.84 ms, Speedup: 0.70x, Differences: 33673
Optical Flow (1024x1024) - CPU: 21.08 ms, GPU: 0.00 ms, Speedup: 5067.31x, Differences: 135594
